# Collate climate and seasonal data fire mask and not v2

In [1]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
date_str = now.strftime("%Y%m%d")
date_time_str = now.strftime("%Y%m%d_%H%M%S")
print(date_str)
print(date_time_str)

20230402
20230402_183649


In [3]:
drive = "D"
met_date = "20230109"
direction = "near"

feature_date = "20230205"

# # processing date
# date = "20230204"

# define output directory
output_dir = r"{0}:\cdu\data\zonal_stats\output\{1}".format(drive, date_str)
#test directory
output_dir = r"D:\cdu\data\zonal_stats\output\test"


# user inputs
v1 = "mean"
v2 = "max"
v3 = "min"
v4 = "med"
v5 = "std"

classify = True
classify = False

climate = True
climate = False

# choose from one or the other to select fire mask applied or notscars or not

# choose for fire mask applied
type_basal_feat_dir = "fire_mask_NOT_applied_basal"
fsm = "fms_applied"

# choose for fire mask NOT applied
# type_basal_feat_dir = "fire_mask_applied_basal"
# fsm = "fms_NOT_applied"



In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from glob import glob
import os

In [5]:
def mk_dir_fn(dir_):
    """ Create a new directory if one does not already exist. """
    if not os.path.isdir(dir_):
        os.mkdir(dir_)

        
def export_csv_fn(list_, dir_, file_name):
    
    """ Create and export path from directory and file name and exports csv with no dropping the index column. """
    
    df_final = pd.concat(list_, axis =0)    
    output_path = os.path.join(dir_, file_name)
    df_final.to_csv(os.path.join(output_path), index=False)
    print("File output to: ", output_path)
    
    return df_final


def glob_concat_axis1_fn(d, s, m):
    """ Concatenate mosaic basal and feature data based on site index. """
    df_list = []

    for f in glob(os.path.join(d, s)):

        df = pd.read_csv(f)
#         df.set_index("uid", inplace = True) # change back to uid once met data has been corrected
        df.set_index("site", inplace = True)
        df_list.append(df)
    
    df_list.append(m)
    # concat data
    df_con = pd.concat(df_list, axis = 1)
    
    # Drop duplicate columns
    df_bm_dd = df_con.T.drop_duplicates().T
    

    return df_list, df_bm_dd


def glob_concat_axis1_fn(d, s, m):
    """ Concatenate mosaic basal and feature data based on site index. """
    df_list = []
    
    
    for f in glob(os.path.join(d, s)):
#         print(f)
        _, file_ = os.path.split(f)
        f_split = file_.split("_")
        
        old_s = f_split[0] 
#         print(old_s)
        new_s = f_split[0] + "_" + f_split[1]
#         print(new_s)

        df = pd.read_csv(f)
        col_list = df.columns.tolist()
        
        s_list = []
        for i in col_list:

            s = i.replace(old_s, new_s)
            s_list.append(s)
            
        df.columns = s_list

        
        # df.set_index("uid", inplace = True) # change back to uid once met data has been corrected
        df.set_index("site", inplace = True)
        df_list.append(df)
    
    df_list.append(m)
    # concat data
    df_con = pd.concat(df_list, axis = 1)
    
    # Drop duplicate columns
    df_bm_dd = df_con.T.drop_duplicates().T
    
    print(s_list)
    return df_list, df_bm_dd


def glob_concat_axis1_no_climate_fn(d, s):
    """ Concatenate mosaic basal and feature data based on site index. """
    df_list = []
    
    print("Searching for: ", os.path.join(d, s))
    for f in glob(os.path.join(d, s)):
        print(f)
        _, file_ = os.path.split(f)
        f_split = file_.split("_")
        
        old_s = f_split[0] 
#         print(old_s)
        new_s = f_split[0] + "_" + f_split[1]
#         print(new_s)

        df = pd.read_csv(f)
        col_list = df.columns.tolist()
        
        s_list = []
        for i in col_list:

            s = i.replace(old_s, new_s)
            s_list.append(s)
            
        df.columns = s_list

        
        # df.set_index("uid", inplace = True) # change back to uid once met data has been corrected
        df.set_index("site", inplace = True)
        df_list.append(df)
    
#     df_list.append(m)
    # concat data
    df_con = pd.concat(df_list, axis = 1)
    
    # Drop duplicate columns
    df_bm_dd = df_con.T.drop_duplicates().T
    
    print(s_list)
    return df_list, df_bm_dd

def identify_null_values(df_in):
    """ Create and export column names and null values as a new df. """
    nc = df_in.isnull().sum()
    df_out = pd.DataFrame(nc[nc > 0].sort_values(ascending=False))
    for index, row in df_out.iterrows():
        print(f"{index}: {row[0]} Null values")

    df_out.reset_index(drop=False, inplace=True)
    
    return df_out


def drop_cols_fn(df, var1, var2, var3, var4, var5):
    """ Select out feature or stats values and export df's """
    df1 = df.copy(deep=True)
    df_columns = df1.columns.tolist()
    
    drop_list = [columns for columns in df_columns if var1 not in columns]  
    drop_list2 = [columns for columns in drop_list if var2 not in columns]
    drop_list3 = [columns for columns in drop_list2 if var3 not in columns]
    drop_list4 = [columns for columns in drop_list3 if var4 not in columns]
    drop_list5 = [columns for columns in drop_list4 if var5 not in columns]
    
    # add fire elements
    drop_list6 = [columns for columns in drop_list5 if "intens" not in columns]
    drop_list7 = [columns for columns in drop_list5 if "fire_f" not in columns]
    drop_list8 = [columns for columns in drop_list7 if "fire_tot" not in columns]
    drop_list9 = [columns for columns in drop_list8 if "since_fire" not in columns]
    drop_list10 = [columns for columns in drop_list9 if "fire_gap" not in columns]
    drop_list11 = [columns for columns in drop_list10 if "burnt" not in columns]
    drop_list12 = [columns for columns in drop_list11 if "fire_pois1_5" not in columns]
    
    #dka fire

    df1.drop(drop_list12, axis = 1, inplace=True)
    var_ = df.iloc[:, [0, 1, 11]]
    df_out = pd.concat([var_, df1], axis=1)
    

    return df1, var_, df_out

##### Create output directories

In [6]:
ml_data_dir = os.path.join(output_dir, "ml_data_dir")
mk_dir_fn(output_dir)
mk_dir_fn(ml_data_dir)

##### Load basal feature zonal stats

In [7]:
bf = r"{0}:\cdu\data\zonal_stats\output\{1}\{2}".format(drive, feature_date, type_basal_feat_dir)
print(bf)

D:\cdu\data\zonal_stats\output\20230205\fire_mask_NOT_applied_basal


##### Load climate data 

In [7]:
if climate:
    print("Climate")
    climate_str = "met"
    m = r"{0}:\cdu\data\zonal_stats\output\{1}\cleaned_{2}_met_zonal_stats.csv".format(drive, met_date, direction)
    print(m)
    m_df = pd.read_csv(m).set_index("site")
    # Call the glob_concat_axis1 function to concatenate features
    bfm_df_list,df_bm_dd = glob_concat_axis1_fn(bf, "*.csv", m_df)

else:
    print("Not Climate")
    # Call the glob_concat_axis1 function to concatenate features
    bfm_df_list,df_bm_dd = glob_concat_axis1_no_climate_fn(bf, "*.csv")
    climate_str = ""

    

Climate
D:\cdu\data\zonal_stats\output\20230109\cleaned_near_met_zonal_stats.csv
['uid', 'site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha', 'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha', 'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'basal_dt', 'uid_dp1_0509_0509', 'image', 's_day', 's_month', 's_year', 's_date', 'e_day', 'e_month', 'e_year', 'e_date', 'b1_dp1_0509_min', 'b1_dp1_0509_max', 'b1_dp1_0509_mean', 'b1_dp1_0509_count', 'b1_dp1_0509_std', 'b1_dp1_0509_med', 'b1_dp1_0509_p25', 'b1_dp1_0509_p50', 'b1_dp1_0509_p75', 'b1_dp1_0509_p95', 'b1_dp1_0509_p99', 'b1_dp1_0509_range', 'b2_dp1_0509_min', 'b2_dp1_0509_max', 'b2_dp1_0509_mean', 'b2_dp1_0509_count', 'b2_dp1_0509_std', 'b2_dp1_0509_med', 'b2_dp1_0509_p25', 'b2_dp1_0509_p50', 'b2_dp1_0509_p75', 'b2_dp1_0509_p95', 'b2_dp1_0509_p99', 'b2_dp1_0509_range', 'b3_dp1_0509_min', 'b3_dp1_0509_max', 'b3_dp1_0509_

In [8]:
df_bm_dd

,uid,date,lon_gda94,lat_gda94,bio_l_kg1ha,bio_t_kg1ha,bio_b_kg1ha,bio_w_kg1ha,bio_br_kg1ha,bio_s_kg1ha,...,evp_s_mean.1,tmax_mean,tmin_mean,rain_m_mean,mslp_mean,rad_mean,rh_tmax_mean,rh_tmin_mean,vp_mean,vp_d_mean
nt001,71,20110523,131.21,-13.9452,0,0,0,0,0,0,...,5.6,32.6,16.3,141.6,1013.9,16.7,3311.1,3368.3,17,22
barkely01,24,20111025,135.041,-18.0018,0,0,0,0,0,0,...,8.4,37,23.6,71.7,1012.3,16.9,3312.2,3353.4,22.4,29.8
barkely04,27,20111026,135.226,-17.9848,0,0,0,0,0,0,...,9,38.6,23.9,72.9,1012.1,19.2,3311.9,3358.1,24.2,31.8
barkely03,26,20111026,135.288,-18.0245,0,0,0,0,0,0,...,8.9,38.5,23.8,73.3,1012.1,19.2,3312.2,3359,24.3,31.4
barkely02,25,20111026,135.26,-18.057,0,0,0,0,0,0,...,9,38.6,23.9,71.5,1012.1,19.2,3311.7,3357.8,24.1,31.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ntadac0002,79,20160506,132.34,-12.7392,662.207,718.914,2368.02,12460.9,5334.93,14828.9,...,5.6,35.1,24.6,228.9,1011.2,18.5,3326.9,3368.7,28.5,20.3
ntaarp0001,75,20160602,132.27,-13.5573,1114.7,1209.66,3360.25,18298.8,7796.61,21659.1,...,5.6,33.1,20.7,203.8,1014.4,19.6,3323.8,3374.4,23.9,18.5
ntaarp0002,76,20160602,132.294,-13.5456,885.524,961.609,3075.57,13171,7308.61,16246.6,...,5.7,32.9,20.7,204.2,1014.5,19.6,3324.1,3374,23.8,18.3
ntaarp0003,77,20160603,132.457,-13.517,628.216,682.37,2271.72,9263.87,5445.67,11535.6,...,6,35.1,21.1,196.1,1014.7,17.7,3318.4,3371.3,23.7,22.8


In [9]:
# m = r"{0}:\cdu\data\zonal_stats\output\{1}\cleaned_{2}_met_zonal_stats.csv".format(drive, met_date, direction)
# print(m)

In [10]:
# m_df = pd.read_csv(m)
# # set index to site or uid
# m_df.set_index("site", inplace = True)
# # m_df

In [11]:
# print(list(m_df))

In [12]:
# # Call the glob_concat_axis1 function to concatenate features
# bfm_df_list,df_bm_dd = glob_concat_axis1_fn(bf, "*.csv", m_df)

# call the drop columns function to select statistical values
taf_df, t_d_df, df_bm_dmmmms = drop_cols_fn(df_bm_dd, v1, v2, v3, v4, v5)


In [13]:
def drop_feature_fn(df):
    """ Remove features from dataframe """
    df1 = df.copy(deep=True)
    df_columns = df.columns.tolist()
    
    drop_list = [columns for columns in df_columns if 'dis_' in columns]
    drop_list2 = [columns for columns in df_columns if 'stc_' in columns]
    drop_list3 = [columns for columns in df_columns if 'mean.1' in columns]

    drop_list.extend(drop_list2)
    drop_list.extend(drop_list3)
    df1.drop(drop_list, axis = 1, inplace=True)
    
        
    return df1
 
    
dmmmms_nodisstc = drop_feature_fn(df_bm_dmmmms)

In [14]:
# dmmmms_nodisstc

In [15]:
def fill_min_list(df):
    """ Fill minimum values with 0 """
    
    df1 = df.copy(deep=True)
    
    df_columns = df.columns.tolist()
    min_list = [columns for columns in df_columns if '_min' in columns]
    
    for i in min_list:
#         print(i)

        df1.dropna(subset=[i], inplace=True)
        mean_value = df1[i].mean()
        print(f"feature {i} mean = {mean_value}")
        df[i].fillna(mean_value, inplace=True)
        
        
        #df[i].fillna((df1[i].mean()), inplace=True) #should be 0
         
    return df
 
    
dmmmms_min = fill_min_list(dmmmms_nodisstc)

feature b1_fpca2_0509_min mean = 6.803360925092326
feature b1_h99a_01122_min mean = 7.438527840335353
feature b1_dbi_0608_min mean = 432.91156462585036
feature b2_dbi_0608_min mean = 684.1088435374149
feature b3_dbi_0608_min mean = 974.0612244897959
feature b4_dbi_0608_min mean = 2121.3401360544217
feature b5_dbi_0608_min mean = 2393.6598639455783
feature b6_dbi_0608_min mean = 1481.6666666666667
feature b1_dja_0305_min mean = 13.863013698630137
feature b1_dim_0305_min mean = 8.582191780821917
feature b2_dim_0305_min mean = 32.83561643835616
feature b3_dim_0305_min mean = 40.71232876712329
feature b1_dim_0608_min mean = 13.746575342465754
feature b2_dim_0608_min mean = 18.95205479452055
feature b3_dim_0608_min mean = 49.31506849315068
feature b1_dim_0911_min mean = 16.438356164383563
feature b2_dim_0911_min mean = 17.883561643835616
feature b3_dim_0911_min mean = 49.06849315068493
feature b1_dim_1202_min mean = 10.413793103448276
feature b2_dim_1202_min mean = 34.282758620689656
featur

In [16]:
# df1 = dmmmms_nodisstc.copy()
# df2 = dmmmms_nodisstc

# df1.dropna(subset=["b1_dja_min"], inplace=True)
# mean_value = df1["b1_dja_min"].mean()
# # print(f"feature {"b1_dja_min"} mean = {mean_value}")
# #dmmmms_nodisstc["b1_dja_min"]= dmmmms_nodisstc["b1_dja_min"].fillna(mean_value, inplace=True)
# df2["b1_dja_min"].fillna(value=mean_value, inplace=True)

  


In [17]:
# print(dmmmms_nodisstc.shape)
# print(df2.shape)

In [18]:
# dmmmms_nodisstc.dropna(inplace=True)
# dmmmms_nodisstc.shape

In [19]:
# df2.dropna(inplace=True)
# df2.shape

In [20]:
# call the identify null values function
df_bm_dmmmms_null = identify_null_values(dmmmms_min)

b3_dp1_0509_std: 22 Null values
b2_dp1_0509_max: 22 Null values
b1_dp1_0509_max: 22 Null values
b1_dp1_0509_mean: 22 Null values
b3_dp1_0509_med: 22 Null values
b1_dp1_0509_med: 22 Null values
b1_dp1_0509_std: 22 Null values
b2_dp1_0509_mean: 22 Null values
b2_dp1_0509_std: 22 Null values
b2_dp1_0509_med: 22 Null values
b3_dp1_0509_max: 22 Null values
b3_dp1_0509_mean: 22 Null values
b1_h99a_01122_max: 20 Null values
b1_h99a_01122_mean: 20 Null values
b1_h99a_01122_med: 20 Null values
b1_h99a_01122_std: 20 Null values
b1_fpca2_0509_mean: 13 Null values
b1_fpca2_0509_med: 13 Null values
b1_fpca2_0509_std: 13 Null values
b1_fpca2_0509_max: 13 Null values
b1_dp1_0112_med: 3 Null values
b3_dp1_0112_max: 3 Null values
b2_dp1_0112_max: 3 Null values
b1_dp1_0112_std: 3 Null values
b2_dp1_0112_mean: 3 Null values
b2_dp1_0112_med: 3 Null values
b2_dp1_0112_std: 3 Null values
b3_dp1_0112_mean: 3 Null values
b3_dp1_0112_med: 3 Null values
b3_dp1_0112_std: 3 Null values
b1_dp1_0112_max: 3 Null val

In [21]:
def mean_null(df):
    """ Mean the null values of data excluding date and biomass."""
    df1 = df.copy(deep=True)

    df_columns = df1.columns.tolist()

    for i in df_columns[3:]:
        df1.dropna(subset=[i], inplace=True)
        mean_value = df1[i].mean()
        print(f"feature {i} mean = {mean_value}")
        df[i].fillna(mean_value, inplace=True)

        
    return df
    

In [22]:
dmmmms_mean = mean_null(dmmmms_min)

feature b1_fpca2_0509_min mean = 6.803360925092326
feature b1_fpca2_0509_max mean = 17.199115225246974
feature b1_fpca2_0509_mean mean = 11.24807630166116
feature b1_fpca2_0509_med mean = 11.001351992805283
feature b1_fpca2_0509_std mean = 2.7999219686105583
feature b1_h99a_01122_min mean = 7.438527840335353
feature b1_h99a_01122_max mean = 12.225477257553411
feature b1_h99a_01122_mean mean = 9.74961009156875
feature b1_h99a_01122_med mean = 9.727706052819077
feature b1_h99a_01122_std mean = 1.278184918888929
feature b1_dbi_0608_min mean = 432.91156462585036
feature b1_dbi_0608_max mean = 561.7210884353741
feature b1_dbi_0608_mean mean = 493.24863945578244
feature b1_dbi_0608_med mean = 491.90816326530614
feature b1_dbi_0608_std mean = 35.03446821031194
feature b2_dbi_0608_min mean = 684.1088435374149
feature b2_dbi_0608_max mean = 863.4013605442177
feature b2_dbi_0608_mean mean = 769.0147108843538
feature b2_dbi_0608_med mean = 766.7482993197278
feature b2_dbi_0608_std mean = 49.38099

In [23]:
dmmmms_mean

,uid,date,bio_agb_kg1ha,b1_fpca2_0509_min,b1_fpca2_0509_max,b1_fpca2_0509_mean,b1_fpca2_0509_med,b1_fpca2_0509_std,b1_h99a_01122_min,b1_h99a_01122_max,...,evp_s_mean,tmax_mean,tmin_mean,rain_m_mean,mslp_mean,rad_mean,rh_tmax_mean,rh_tmin_mean,vp_mean,vp_d_mean
nt001,71,20110523,0,13.779837,46.998966,25.849381,23.867432,8.477655,7.273514,11.780785,...,5.2,32.6,16.3,141.6,1013.9,16.7,3311.1,3368.3,17.0,22.0
barkely01,24,20111025,0,0.752944,2.872306,1.691590,1.486260,0.699975,4.384300,8.969109,...,8.7,37.0,23.6,71.7,1012.3,16.9,3312.2,3353.4,22.4,29.8
barkely04,27,20111026,0,2.120632,6.417120,4.374468,4.472551,1.216005,5.932899,8.854515,...,7.8,38.6,23.9,72.9,1012.1,19.2,3311.9,3358.1,24.2,31.8
barkely03,26,20111026,0,0.752944,3.292108,1.820417,1.637530,0.699339,3.664345,7.094025,...,7.8,38.5,23.8,73.3,1012.1,19.2,3312.2,3359.0,24.3,31.4
barkely02,25,20111026,0,2.120632,13.779837,4.974984,4.725986,2.433672,3.129379,5.818588,...,7.9,38.6,23.9,71.5,1012.1,19.2,3311.7,3357.8,24.1,31.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ntadac0002,79,20160506,20805.2,15.538734,42.389004,29.117230,29.265415,6.836081,16.053879,20.420395,...,5.7,35.1,24.6,228.9,1011.2,18.5,3326.9,3368.7,28.5,20.3
ntaarp0001,75,20160602,30472.4,27.614885,38.921173,34.748572,34.327137,3.238761,17.312332,21.588921,...,6.0,33.1,20.7,203.8,1014.4,19.6,3323.8,3374.4,23.9,18.5
ntaarp0002,76,20160602,24414.1,9.056872,22.308666,14.395230,14.214185,3.600052,9.738800,16.808601,...,6.0,32.9,20.7,204.2,1014.5,19.6,3324.1,3374.0,23.8,18.3
ntaarp0003,77,20160603,17598.4,5.261469,25.453907,13.082606,12.109756,5.789599,9.613670,20.052162,...,6.3,35.1,21.1,196.1,1014.7,17.7,3318.4,3371.3,23.7,22.8


In [24]:
# call the identify null values function
df_bm_dmmmms_null = identify_null_values(dmmmms_mean)

In [25]:
dmmmms_mean.dropna(inplace=True)

In [26]:
dmmmms_mean.sample(20)

,uid,date,bio_agb_kg1ha,b1_fpca2_0509_min,b1_fpca2_0509_max,b1_fpca2_0509_mean,b1_fpca2_0509_med,b1_fpca2_0509_std,b1_h99a_01122_min,b1_h99a_01122_max,...,evp_s_mean,tmax_mean,tmin_mean,rain_m_mean,mslp_mean,rad_mean,rh_tmax_mean,rh_tmin_mean,vp_mean,vp_d_mean
jdr03,47,20120523,11180.4,10.533810,18.330997,14.593527,14.648533,1.912195,9.042366,15.288635,...,6.4,32.5,17.4,202.0,1013.3,18.6,3316.2,3374.2,19.4,20.0
auv05,19,20121011,73.92,0.275141,1.788801,1.025849,0.968540,0.445413,5.553467,8.010284,...,9.7,36.3,25.0,135.8,1013.3,24.4,3313.8,3347.5,22.5,29.2
amg06,5,20120621,3086.16,8.356935,23.343081,16.552498,17.380726,3.837718,8.267647,15.495010,...,4.5,27.3,5.6,101.5,1017.4,18.9,3302.1,3376.5,9.3,16.9
amg03,3,20120620,0,3.741091,13.779837,6.702390,6.417120,2.946635,5.320847,8.553528,...,4.7,26.9,6.7,96.4,1018.6,19.5,3296.0,3346.8,6.9,19.3
ntagfu0015,94,20120514,1921,3.292108,8.356935,4.909667,4.219115,1.356020,4.595810,8.722031,...,8.1,30.2,15.9,79.8,1017.8,19.9,3301.7,3336.3,10.8,24.1
ntamgd0001,121,20140313,5778.08,6.803361,17.199115,11.248076,11.001352,2.799922,7.438528,12.225477,...,8.8,35.4,22.9,87.3,1012.0,24.1,3303.8,3332.7,15.7,32.6
barkely03,26,20111026,0,0.752944,3.292108,1.820417,1.637530,0.699339,3.664345,7.094025,...,7.8,38.5,23.8,73.3,1012.1,19.2,3312.2,3359.0,24.3,31.4
amg08,6,20120622,29521.8,23.343081,45.850407,35.404507,35.469837,5.048976,11.452576,18.281776,...,4.8,27.0,5.0,97.9,1015.9,19.3,3301.5,3376.5,8.9,16.7
ntamgd0005,125,20160429,5117.8,2.872306,16.449720,11.028107,12.109756,4.345881,8.132851,13.228630,...,6.5,34.6,23.8,237.5,1013.9,18.7,3320.9,3359.3,24.4,22.9
ntagfu0032,111,20120707,2474.01,1.212895,12.109756,3.387322,2.677053,2.383302,3.381958,8.623528,...,4.6,28.6,13.8,134.4,1014.0,19.5,3308.4,3355.8,12.5,19.0


In [27]:
class_list = []

if classify:
    str_out= "class"
    for i in dmmmms_mean.bio_agb_kg1ha:
        if i == 0:
            x = 0
        elif i > 0 and i <1000:
            x = 1
#         elif i >= 500 and i < 1000:
#             x = 2
        elif i >= 1000 and i < 2000:
            x = 2
        elif i >=2000 and i < 3000:
            x = 3
        elif i >=3000 and i < 4000:
            x = 4
        elif i >=4000 and i < 5000:
            x = 5
        elif  i >=5000 and i < 10000:
            x = 6 
        elif i >= 10000 and i < 20000:
            x = 7
        elif i >= 20000 and i < 30000:
            x = 8
        elif i >=30000 and i < 40000:
            x = 9
        elif i >=40000 and i < 50000:
            x = 10
        else:
            x = 11
            
        print(f"class {x}: {i}")
        class_list.append(x)
    dmmmms_mean["target"] = class_list
    
else:
    str_out = "reg"
    dmmmms_mean["target"] = dmmmms_mean.bio_agb_kg1ha
    

dmmmms_mean.drop("bio_agb_kg1ha", axis=1, inplace=True)    

class 0: 0.0
class 0: 0.0
class 0: 0.0
class 0: 0.0
class 0: 0.0
class 2: 1894.2
class 0: 0.0
class 0: 0.0
class 2: 1536.15
class 2: 1164.24
class 5: 4736.0133333333315
class 0: 0.0
class 2: 1549.086
class 0: 0.0
class 0: 0.0
class 0: 0.0
class 0: 0.0
class 2: 1947.5866666666723
class 2: 1293.6
class 3: 2540.846
class 2: 1796.666666666668
class 3: 2357.226666666664
class 2: 1920.996
class 3: 2608.7599999999993
class 3: 2833.213250000001
class 0: 0.0
class 3: 2389.599333333333
class 0: 0.0
class 8: 21469.14
class 8: 22699.39
class 0: 0.0
class 4: 3742.2
class 7: 11180.4
class 0: 0.0
class 0: 0.0
class 0: 0.0
class 5: 4872.776999999999
class 6: 5588.142
class 4: 3156.902
class 6: 8031.45
class 6: 7658.5425
class 7: 17994.759999999995
class 4: 3123.12
class 6: 7711.293333333332
class 9: 35199.71
class 4: 3915.4499999999935
class 5: 4370.912
class 3: 2215.29
class 0: 0.0
class 6: 7203.195999999995
class 1: 689.9199999999995
class 4: 3615.832500000005
class 6: 6044.738
class 6: 5606.4384444

In [28]:
dmmmms_mean.reset_index(drop=False, inplace=True)
dmmmms_mean.rename(columns={"index": "site"}, inplace=True)
dmmmms_mean.dropna(inplace=True)

In [29]:
dmmmms_mean

,site,uid,date,b1_fpca2_0509_min,b1_fpca2_0509_max,b1_fpca2_0509_mean,b1_fpca2_0509_med,b1_fpca2_0509_std,b1_h99a_01122_min,b1_h99a_01122_max,...,tmax_mean,tmin_mean,rain_m_mean,mslp_mean,rad_mean,rh_tmax_mean,rh_tmin_mean,vp_mean,vp_d_mean,target
0,nt001,71,20110523,13.779837,46.998966,25.849381,23.867432,8.477655,7.273514,11.780785,...,32.6,16.3,141.6,1013.9,16.7,3311.1,3368.3,17.0,22.0,0
1,barkely01,24,20111025,0.752944,2.872306,1.691590,1.486260,0.699975,4.384300,8.969109,...,37.0,23.6,71.7,1012.3,16.9,3312.2,3353.4,22.4,29.8,0
2,barkely04,27,20111026,2.120632,6.417120,4.374468,4.472551,1.216005,5.932899,8.854515,...,38.6,23.9,72.9,1012.1,19.2,3311.9,3358.1,24.2,31.8,0
3,barkely03,26,20111026,0.752944,3.292108,1.820417,1.637530,0.699339,3.664345,7.094025,...,38.5,23.8,73.3,1012.1,19.2,3312.2,3359.0,24.3,31.4,0
4,barkely02,25,20111026,2.120632,13.779837,4.974984,4.725986,2.433672,3.129379,5.818588,...,38.6,23.9,71.5,1012.1,19.2,3311.7,3357.8,24.1,31.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,ntadac0002,79,20160506,15.538734,42.389004,29.117230,29.265415,6.836081,16.053879,20.420395,...,35.1,24.6,228.9,1011.2,18.5,3326.9,3368.7,28.5,20.3,8
163,ntaarp0001,75,20160602,27.614885,38.921173,34.748572,34.327137,3.238761,17.312332,21.588921,...,33.1,20.7,203.8,1014.4,19.6,3323.8,3374.4,23.9,18.5,9
164,ntaarp0002,76,20160602,9.056872,22.308666,14.395230,14.214185,3.600052,9.738800,16.808601,...,32.9,20.7,204.2,1014.5,19.6,3324.1,3374.0,23.8,18.3,8
165,ntaarp0003,77,20160603,5.261469,25.453907,13.082606,12.109756,5.789599,9.613670,20.052162,...,35.1,21.1,196.1,1014.7,17.7,3318.4,3371.3,23.7,22.8,7


In [30]:
# dmmmms_mean.to_csv(os.path.join(ml_data_dir, f"season_met_fillna_mean_{fsm}_applied.csv"))

In [31]:
path_ = os.path.join(ml_data_dir, f"s_{v1}_{v2}_{v3}_{v4}_{v5}_{climate_str}_fnm_{fsm}_{str_out}.csv")      
dmmmms_mean.to_csv(path_, index=False)
print(path_)

D:\cdu\data\zonal_stats\output\20230205\ml_data_dir\s_mean_max_min_med_std_met_fnm_fms_applied_class.csv


In [32]:
def drop_low_r2_seasons_fn(df):
    """ Remove features from dataframe """
    df_columns = df.columns.tolist()
    
    drop_list = [columns for columns in df_columns if '0305' in columns]
    drop_list2 = [columns for columns in df_columns if '1202' in columns]
    drop_list3 = [columns for columns in df_columns if 'dp1_0509' in columns]
    
    drop_list.extend(drop_list2)
    drop_list.extend(drop_list3)
    df.drop(drop_list, axis = 1, inplace=True)
    
        
    return df
 
    
selected_features_and_seasons = drop_feature_fn(dmmmms_mean)

In [33]:
path_ = os.path.join(ml_data_dir, f"r2_bs_{v1}_{v2}_{v3}_{v4}_{v5}_{climate_str}_fnm_{fsm}_{str_out}.csv")      
selected_features_and_seasons.to_csv(path_, index=False)
print(path_)

D:\cdu\data\zonal_stats\output\20230205\ml_data_dir\r2_bs_mean_max_min_med_std_met_fnm_fms_applied_class.csv


In [34]:
import seaborn as sns
sns.histplot(data=dmmmms_mean, x="target")

<AxesSubplot:xlabel='target', ylabel='Count'>